In [1]:
import tensorflow as tf
import numpy as np

In [2]:
learning_rate = 0.1

In [3]:
x_data = np.array( [
    [ 0, 0 ],
    [ 0, 1 ],
    [ 1, 0 ],
    [ 1, 1 ],
], dtype = np.float32 )

y_data = np.array( [
    [ 0 ],
    [ 1 ],
    [ 1 ],
    [ 0 ]
], dtype = np.float32 )

In [4]:
W1 = tf.Variable( tf.random.normal( [ 2, 2 ] ), name = 'weight1' )
b1 = tf.Variable( tf.random.normal( [ 2 ] ), name = 'bias' )
l1 = lambda X: ( tf.math.sigmoid( tf.matmul( X, W1 ) + b1 ) )

W2 = tf.Variable( tf.random.normal( [ 2, 1 ] ), name = 'weight2' )
b2 = tf.Variable( tf.random.normal( [ 1 ] ), name = 'bias2' )
Y_pred = lambda X: ( tf.math.sigmoid( tf.matmul( l1( X ), W2 ) + b2 ) )

In [5]:
def Cost( X, Y ):
    y_pred = Y_pred( X )
    
    return ( -tf.reduce_mean( Y * tf.math.log( y_pred) + ( 1. - Y) *
                            tf.math.log( 1. - y_pred ) ) )

In [6]:
def d_Y_pred( X, Y ):
    y_pred = Y_pred( X )
    return ( y_pred - Y) / (y_pred * (1.0 - y_pred) + 1e-7)

d_sigma2 = lambda X: Y_pred( X ) * (1 - Y_pred( X ) )
d_a2 = lambda X, Y: d_Y_pred( X, Y ) * d_sigma2( X )
d_p2 = lambda X, Y: d_a2( X, Y )
d_b2 = lambda X, Y: d_a2( X, Y )
d_W2 = lambda X, Y: tf.matmul( tf.transpose( l1( X ) ), d_p2( X, Y ) )

d_b2_mean = lambda X, Y: tf.reduce_mean( d_b2( X, Y ), axis = [ 0 ] )
d_W2_mean = lambda X, Y: d_W2( X, Y ) / tf.cast( tf.shape( l1( X ) )[ 0 ],
                                                dtype = tf.float32)

d_l1 = lambda X, Y: tf.matmul( d_p2( X, Y ), tf.transpose( W2 ) )
d_sigma1 = lambda X: l1( X ) * (1 - l1( X ) )
d_a1 = lambda X, Y: d_l1( X, Y ) * d_sigma1( X )
d_b1 = lambda X, Y: d_a1( X, Y )
d_p1 = lambda X, Y: d_a1( X, Y )
d_W1 = lambda X, Y: tf.matmul( tf.transpose( X ), d_a1( X, Y ) )

d_W1_mean = lambda X, Y: d_W1( X, Y ) / tf.cast( tf.shape( X )[ 0 ],
                                                dtype = tf.float32 )
d_b1_mean = lambda X, Y: tf.reduce_mean( d_b1( X, Y ), axis = [ 0 ] )

In [7]:
def Step( X, Y ):
    return ( [
    W2.assign( W2 - learning_rate * d_W2_mean( X, Y ) ),
    b2.assign( b2 - learning_rate * d_b2_mean( X, Y ) ),
    W1.assign( W1 - learning_rate * d_W1_mean( X, Y ) ),
    b1.assign( b1 - learning_rate * d_b1_mean( X, Y ) )
] )

predicted = lambda X: tf.cast( Y_pred( X ) > 0.5, dtype = tf.float32 )
accuracy = lambda X, Y: tf.reduce_mean( tf.cast( tf.equal( predicted( X ), Y ),
                                               dtype = tf.float32 ) )

In [8]:
X = x_data
Y = y_data

print( 'shape: ', tf.shape( X )[0] )
for i in range( 10001 ):
    Step( X, Y )
    if i % 1000 == 0:
        tf.print(i,
                Cost( X, Y ), d_W1( X, Y ), W1, W2 )

shape:  tf.Tensor(4, shape=(), dtype=int32)
0 1.29891086 [[-0.112265483 0.229306221]
 [-0.155986741 0.102845639]] [[-1.42160964 -0.863124669]
 [-0.797892332 -1.92852497]] [[0.871810079]
 [-1.46809757]]
1000 0.514564872 [[0.0327383131 0.0845496729]
 [0.0530274659 0.0714549422]] [[-1.12723076 -3.16769791]
 [-0.888684034 -3.25292754]] [[1.98727584]
 [-3.58789968]]
2000 0.229348242 [[0.0574156344 0.0389345251]
 [0.058300525 0.0388754942]] [[-2.48450971 -4.67995691]
 [-2.47961617 -4.6781826]] [[4.47816]
 [-5.93455553]]
3000 0.0925777 [[0.0272065774 0.0178905595]
 [0.0272213146 0.017912792]] [[-3.49942923 -5.34190416]
 [-3.49910879 -5.34059954]] [[6.47233725]
 [-7.44846392]]
4000 0.0522871166 [[0.0152419619 0.0105930669]
 [0.0152442604 0.0106031289]] [[-4.00631618 -5.68421602]
 [-4.00614882 -5.68328524]] [[7.59995127]
 [-8.37756252]]
5000 0.0354705341 [[0.0101134107 0.00730945729]
 [0.0101144165 0.00731512159]] [[-4.31553698 -5.90342379]
 [-4.31540442 -5.90267754]] [[8.3413105]
 [-9.02240276

In [9]:
h, c, a = Y_pred( X ), predicted( X ), accuracy( X, Y )
tf.print( '\nHypothesis: ', h, '\nCorrect: ', c, '\nAccuracy: ', a )


Hypothesis:  [[0.0104005076]
 [0.988399744]
 [0.988400221]
 [0.0178418942]] 
Correct:  [[0]
 [1]
 [1]
 [0]] 
Accuracy:  1
